In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Above code gives list of files and directories

In [ ]:
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import cv2
import glob
import os

# above code imports 3 pakages

In [ ]:
os.mkdir('fm')

In [ ]:
os.mkdir('fm/mask')

In [ ]:
os.mkdir('fm/nomask')

In [ ]:
inputFolder='../input/covid-face-mask-detection-dataset/New Masks Dataset/Train/Non Mask'
folderLen= len(inputFolder)
#os.mkdir('./fm/mask')
for img in glob.glob(inputFolder +"/*.*"):
    image=cv2.imread(img)
    imgResized=cv2.resize(image,(150,150))
    cv2.imwrite("fm/nomask"+img[folderLen:],imgResized)

In [ ]:
inputFolder='../input/covid-face-mask-detection-dataset/New Masks Dataset/Train/Mask'
folderLen= len(inputFolder)
#os.mkdir('./fm/nomask')
for img in glob.glob(inputFolder +"/*.*"):
    image=cv2.imread(img)
    imgResized=cv2.resize(image,(150,150))
    cv2.imwrite("fm/mask"+img[folderLen:],imgResized)

In [ ]:
inputFolder='./fm/nomask'
folderLen= len(inputFolder)
#os.mkdir('fm')
for img in glob.glob(inputFolder +"/*.*"):
    image=cv2.imread(img)
    print(image.shape)

In [ ]:
inputFolder='./fm/mask'
folderLen= len(inputFolder)
#os.mkdir('fm')
for img in glob.glob(inputFolder +"/*.*"):
    image=cv2.imread(img)
    print(image.shape)

# Designing CNN

STEP 1- IMPORTING LIBRARY

In [ ]:
#IMPORTING LIBRARIES
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

STEP 2 PREPARING TRAINING SET

In [ ]:
#train_data is used for feature scaling and image augmentation (image augmentation is applied to avoid overfitting).
train_data = ImageDataGenerator(rescale = 1./255,shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

#defining training set, here size of image is reduced to 64x64, batch of images is kept as 64 and class is defined as 'binary'.
training_set = train_data.flow_from_directory('./fm', batch_size = 5, target_size = (150,150), class_mode = 'binary')

STEP 3 DESIGNING CNN

In [ ]:
#defining the CNN as a sequence of layers.
cnn = tf.keras.models.Sequential()

In [ ]:
#adding 1st Convolutional layer
#note that in image augmentation we kept the image size as 64x64, therefore input_shape should also be same [64,64,3] (here 3 signifies that this is a colorful image (R,G,B))
cnn.add(tf.keras.layers.Conv2D(filters = 32,kernel_size = 3, input_shape = [150,150,3],activation = 'relu'))
#activation function relu is applied to decrease any linearity that might have arrised while applying filters.

In [ ]:
# applying max pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

In [ ]:
#adding 2nd Convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters = 32,kernel_size = 3, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

In [ ]:
#adding 3rd Convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters = 32,kernel_size = 3, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

In [ ]:
#the input of step 4 is an flattened array,
cnn.add(tf.keras.layers.Flatten())

In [ ]:
#forming an ann with 128 input neurons
cnn.add(tf.keras.layers.Dense(units = 300, activation = 'relu'))

In [ ]:
#adding ouput layer of the ann
cnn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

In [ ]:
#compiling the CNN 
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

STEP 4 - GIVING TRAINING DATA TO CNN FOR LEARNING

In [ ]:
#training the model
cnn.fit(x = training_set,  epochs = 50)

In [ ]:
cnn.save('facemask_cnn_model.h5')

In [ ]:
from keras.models import load_model 
import numpy as np
from keras.preprocessing import image

def test_mask(model,testimage):
    classifier = load_model('facemask_cnn_model.h5')
    training_set.class_indices
    #importing images
    test_img = image.load_img(testimage,target_size = (150,150))
    #converting image to array
    test_img = image.img_to_array(test_img)
    test_img = np.expand_dims(test_img,axis = 0)
    result = classifier.predict(test_img)
    if result[0][0] >= 0.5:
        prediction = ' Picture without mask'
    else:
        prediction = 'Picture with mask'
    return prediction
    

In [ ]:
test_mask('facemask_cnn_model.h5','./fm/nomask/259.jpg')

In [ ]:
from keras.models import load_model 
classifier = load_model('facemask_cnn_model.h5')

In [ ]:
import numpy as np
from keras.preprocessing import image
training_set.class_indices

In [ ]:
testimage=input("Link of test Image")

In [ ]:
image.load_img(testimage)


In [ ]:
#importing images
test_img = image.load_img(testimage,target_size = (150,150))
#converting image to array
test_img = image.img_to_array(test_img)
test_img = np.expand_dims(test_img,axis = 0)
result = classifier.predict(test_img)
if result[0][0] >= 0.5:
    prediction = ' Picture without mask'
else:
    prediction = 'Picture with mask'
print(prediction)

In [ ]:
image.load_img('./fm/mask/0040.jpg')

In [ ]:
#image.load_img('./fm/mask/face (3).jpeg')

In [ ]:
#importing images
test_img = image.load_img('./fm/mask/0040.jpg',target_size = (150,150))
#converting image to array
test_img = image.img_to_array(test_img)
test_img = np.expand_dims(test_img,axis = 0)
result = classifier.predict(test_img)
if result[0][0] >= 0.5:
    prediction = ' Picture without mask'
else:
    prediction = 'Picture with mask'
print(prediction)

In [ ]:
#directry has been removed showing error
#image.load_img('./IMG20210712200636_00.jpg')

In [ ]:
#importing images
test_img = image.load_img('./IMG20210712200636_00.jpg',target_size = (150,150))
#converting image to array
test_img = image.img_to_array(test_img)
test_img = np.expand_dims(test_img,axis = 0)
result = classifier.predict(test_img)
if result[0][0] >= 0.5:
    prediction = ' Picture without mask'
else:
    prediction = 'Picture with mask'
print(prediction)

In [ ]:
inputFolder=input("Link of Folder")

In [ ]:
for img in glob.glob(inputFolder +"/*.*"):
    print(img)
    image.load_img(img)
    #importing images
    test_img = image.load_img(img,target_size = (150,150))
#converting image to array
    test_img = image.img_to_array(test_img)
    test_img = np.expand_dims(test_img,axis = 0)
    result = classifier.predict(test_img)
    if result[0][0] >= 0.5:
        prediction = ' Picture without mask'
    else:
        prediction = 'Picture with mask'
    print(prediction)

In [ ]:
 image.load_img(img)